In [ ]:
#%load_ext autoreload

In [ ]:
# from SeqTransformer import *

# input_dim = 198

# model = build_seq_transformer(input_dim)

# a_input = torch.randn(5, 15, input_dim)
# b_input = torch.randn(5, 10, input_dim)
# output = model(a_input, b_input)

# output[0].shape

In [ ]:
from SeqTransformer import *

input_dim = 198

model = build_seq_vae(input_dim, 2)

In [ ]:
a_input = torch.randn(4, 16, input_dim)
b_input = torch.randn(4, 12, input_dim)
output,_ = model(a_input, b_input)

In [ ]:
output.shape

In [ ]:
model.encoder.config

In [ ]:
import numpy as np
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(params)

----------------------------------------

In [ ]:
from MotionGPT import *

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # device

In [ ]:
loader = FBXDataLoader("FBXDataStand//", has_translate=True)

In [ ]:
loader.LoadData()

In [ ]:
#loader.raw_files

In [ ]:
print(len(loader.raw_data),len(loader.raw_data[0]),len(loader.raw_data[0][0]),len(loader.raw_data[0][0][0]))

In [ ]:
count_length = 0
for item in loader.raw_data:
    count_length += len(item)
print(count_length)

In [ ]:
36622 / 24

In [ ]:
loader.translate_scale

In [ ]:
loader.PrepareTrainingData(training_joint_index=[i for i in range(len(G_MIXAMO_JOINTS))])

In [ ]:
print(len(loader.all_data),len(loader.all_data[0]),len(loader.all_data[0][0]))
print(len(loader.all_data),len(loader.all_data[-1]),len(loader.all_data[-1][0]))

In [ ]:
print(len(loader.train_data),len(loader.train_data[0]),len(loader.train_data[0][1]))

In [ ]:
#build model for training
if torch.cuda.is_available():
    model = model.cuda()

optim = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
epochs = 1

In [ ]:
for epoch in range(epochs): 
    for batch_x, batch_mask in loader.next_batch(batch_size=4):
        if len(batch_x[0]) < 2:
            continue

        batch_x = batch_x.to(device)
        batch_mask = batch_mask.to(device)

        batch_x_decoder_input = batch_x[:,1:, :] # Batch x (Length - 1) x Dim
        batch_x_encoder_input = batch_x[:,:-1, :] # Batch x (Length - 1) x Dim

        batch_mask_encoder_input = batch_mask[:, 1:] # Batch x (Length - 1)

        batch_output, gaussian_params = model(batch_x_encoder_input, batch_x_decoder_input)
        
        mse_loss = calculate_mse_loss(batch_output, batch_x_decoder_input, batch_mask_encoder_input)

        kld_loss = calculate_kld_gauss(*gaussian_params, batch_mask_encoder_input)    

        total_loss = mse_loss + kld_loss

        optim.zero_grad()
        total_loss.backward()
        optim.step()
        
        break

        print(total_loss)
                                    

In [ ]:
mse_loss

In [ ]:
batch_mask.shape

In [ ]:
torch.sum(batch_mask)

In [ ]:
batch_x.shape

In [ ]:
torch.max(batch_x_decoder_input)

In [ ]:
from Params import *
from VRNN import *

In [ ]:
input_dim = len(loader.train_data[0][1])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # device

print("Training data dim:", input_dim)
print("Training on device: ", device)
#Build model
model2 = VRNN(input_dim, h_dim, z_dim, n_layers, device)
model2.to(device)
optimizer = torch.optim.Adam(model2.parameters(), lr=learning_rate)


In [ ]:
epochs = 1

In [ ]:
#Train
for epoch in range(epochs):
    batch_idx = 0
    model.train()
    for batch_data, pad_data in loader.next_batch(batch_size=4, batch_first=False):
        batch_idx += 1

        batch_data = batch_data.to(device)
        pad_data = pad_data.to(device)

        optimizer.zero_grad()

        kld_loss, mse_loss, _, _ = model2(batch_data, pad_data)
        #kld_loss = kld_loss / torch.sum(pad_data)
        #mse_loss = mse_loss / torch.sum(pad_data)

        #print(torch.sum(pad_data))
        #print(torch.sum(pad_data))

        loss = kld_loss + mse_loss
        loss.backward()

        # grad norm clipping, only in pytorch version >= 1.10
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        break

In [ ]:
batch_data.shape

In [ ]:
torch.max(batch_data)

In [ ]:
mse_loss

In [ ]:
pad_data.shape

In [ ]:
a = torch.randn(4,14,198)
b = torch.randn(4,14,198)
c = torch.empty(4, 14).random_(2)

In [ ]:
calculate_mse_loss(a, b, c)

In [ ]:
a1 = a[:,0,:]
b1 = b[:,0,:]
c1 = c[:,0]